In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import TargetEncoder

dataset = pd.read_csv('datasets/dataset_cleaned.csv')
# dataset = pd.get_dummies(dataset, columns=['STATE'], dtype=int)
# dataset.head()

dataset['STATE'] = TargetEncoder(smooth='auto').fit_transform(pd.DataFrame(dataset['STATE']),   dataset['TOTAL_FIRE_SIZE'])

X = dataset.drop('TOTAL_FIRE_SIZE', axis=1)
y = dataset['TOTAL_FIRE_SIZE']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the XGBoost classifier
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

def calc_log_clamped_score(y_true, y_pred):
    """
    Implements: mean( min( |log(y_pred / y_true)|, 10 ) )
    """
    eps = 1e-15
    y_pred = np.maximum(y_pred, eps)
    y_true = np.maximum(y_true, eps)

    log_errors = np.abs(np.log(y_pred / y_true))
    log_errors_clamped = np.minimum(log_errors, 10.0)
    return np.mean(log_errors_clamped)


# Make predictions and calculate mean squared error
y_pred = model.predict(X_test)
score = calc_log_clamped_score(y_test, y_pred)
print("Score:", score)

Score: 3.811559011023252


In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import TargetEncoder

# Load the dataset
dataset = pd.read_csv('datasets/dataset_cleaned.csv')

# Encode 'STATE' using a target encoder
encoder = TargetEncoder()
dataset['STATE'] = encoder.fit_transform(dataset['STATE'], dataset['TOTAL_FIRE_SIZE'])

# Prepare features and target
X = dataset.drop('TOTAL_FIRE_SIZE', axis=1).values
y = dataset['TOTAL_FIRE_SIZE'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the custom loss function
def custom_loss(y_true, y_pred):
    eps = 1e-15
    y_pred = tf.maximum(y_pred, eps)
    y_true = tf.maximum(y_true, eps)
    log_errors = tf.abs(tf.math.log(y_pred / y_true))
    log_errors_clamped = tf.minimum(log_errors, 10.0)
    return tf.reduce_mean(log_errors_clamped)

# Build a simple neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Linear activation by default for regression
])

# Compile the model with the custom loss
model.compile(optimizer='adam', loss=custom_loss)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Make predictions on the test set
y_pred = model.predict(X_test).flatten()

# Calculate the custom score using numpy (for verification)
def calc_log_clamped_score_np(y_true, y_pred):
    eps = 1e-15
    y_pred = np.maximum(y_pred, eps)
    y_true = np.maximum(y_true, eps)
    log_errors = np.abs(np.log(y_pred / y_true))
    log_errors_clamped = np.minimum(log_errors, 10.0)
    return np.mean(log_errors_clamped)

score = calc_log_clamped_score_np(y_test, y_pred)
print("Score:", score)

2025-02-22 13:42:09.451060: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 13:42:09.451716: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 13:42:09.454068: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 13:42:09.459625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740228129.468717  706813 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740228129.47

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.